In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB

corpus = pd.read_csv('corpus.csv.gz', compression='gzip')
posts = pd.read_csv('posts_count_me_pol.csv.gz', compression='gzip', error_bad_lines=False)
stopwords = stopwords.words("portuguese")
feature_names = pd.read_csv('feature_names.csv')

posts = posts[posts['me'] > 2]
posts = posts[posts['polarity'] > 2]
posts = posts.reset_index()
posts.shape

In [ ]:
corpus = pd.read_csv('corpus.csv.gz', compression='gzip')
corpus = corpus[corpus['qual_a_melhor_classificao_para_esse_texto:confidence'] == 1]
corpus = corpus.reset_index()
corpus.shape

In [ ]:
# fix labels to binary
lb = preprocessing.LabelBinarizer(neg_label=1, pos_label=2)
target = lb.fit_transform(corpus['qual_a_melhor_classificao_para_esse_texto'].values)
c, r = target.shape
target = target.reshape(c,)
corpus['class'] = target

In [ ]:
corpus[['qual_a_melhor_classificao_para_esse_texto','class']].sample(5)

In [ ]:
posts['class'] = 0
column_idx = posts.columns.get_loc('class')

for idx_corpus in corpus.index:
    postid = corpus.iloc[idx_corpus].postid
    idx_post = posts[posts['postID']==postid].index
    posts.iloc[idx_post,column_idx] = corpus.iloc[idx_corpus]['class']

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,1), strip_accents='unicode', 
                             stop_words=stopwords, vocabulary=feature_names['0'].values)
data = vectorizer.fit_transform(posts.content)
data.shape

In [ ]:
matrixPD = pd.DataFrame(data.toarray())
matrixPD['class'] = posts['class']
matrixPD = matrixPD[matrixPD['class'] > 0]
matrixPD.shape

label = matrixPD['class'].values
features = matrixPD.drop('class',axis=1)

In [ ]:
from sklearn.model_selection import cross_val_score
model = MultinomialNB()
model.fit(features, label)
cross_val_score(model, features, label, cv=10, scoring='accuracy').mean()

In [ ]:
posts['prediction'] = model.predict(data)

In [ ]:
storyPosts = posts[posts['prediction']==1]
blogs = storyPosts[['posts.csv']].groupby(['posts.csv'])
autores = storyPosts[['authorID']].groupby(['authorID'])
print('Postagens: %i' %len(storyPosts))
print('Blogs: %i' %len(blogs))
print('Autores: %i' %len(autores))

In [ ]:
storyPosts['blogID'] = storyPosts['posts.csv'].astype(int).astype('str')
groupReflex = storyPosts[['blogID','contentCount']].groupby(['blogID']).agg(['count','mean'])
groupReflex['contentCount'].sort_values('count',ascending=False).head(10)

In [ ]:
len(groupReflex[groupReflex['contentCount']['count'] >= 6])

In [ ]:
groupReflex[groupReflex['contentCount']['count'] >= 6].sum()